# Game prototype

In [1]:
from collections.abc import Callable
from abc import ABC, abstractmethod
from dataclasses import dataclass
from typing import TypeVar, Type

from collections import Counter
from IPython.display import Image, clear_output

#TODO: ask niklas what are the three different image types
from IPython.display import Image as IPImage
from pathlib import Path
from PIL import Image as PImage
from openai import OpenAI
import dspy
import requests

import base64
from io import BytesIO

import logging
import regex

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(name)s: %(message)s",
    filename="app.log",
    filemode="a",
)

CanvasType = Image | str
QueryType = CanvasType | list[CanvasType]
AlphabetType = dict[str, QueryType] | list[str]  # first str index/name of symbol

In [2]:
global_buffer = None

In [16]:
# ───────────────────────────────────
# EXAMPLE & TASK
# ───────────────────────────────────


@dataclass
class Example:
    """
    One example puzzle: contains the message to encode,
    the allowed alphabets for encoding, and the alphabet for decoding.
    """

    def __init__(
        self,
        message: QueryType,
        encoder_alphabet: AlphabetType,
        decoder_alphabet: AlphabetType,
        validator_alphabet: AlphabetType,
    ):
        self.message = message
        self.encoder_alphabet = encoder_alphabet
        self.decoder_alphabet = decoder_alphabet
        self.validator_alphabet = validator_alphabet


@dataclass()
class Task:
    """A set of examples forming a complete task."""

    def __init__(
        self,
        examples: list[Example],
        separator: chr,
        description_format: tuple[str, str, str, str],
    ):
        self.examples = examples
        self.separator = separator  # TODO: empty "" for single symbols over alphabet
        (
            self.message_category,
            self.encoder_alphabet_name,
            self.decoder_alphabet_name,
            self.validator_alphabet_name,
        ) = description_format
        """self.encode_description = "\nGiven a {message_category} as query, encode (not necessarily transcribe) it with symbols out of {encoder_alphabet_name}(alphabet). Divide symbols by: '{separator}'".format(
            message_category=self.message_category,
            encoder_alphabet_name=self.encoder_alphabet_name,
            separator=self.separator,
        )"""

        self.encode_description = "\nGiven a {message_category} as query, encode (not necessarily transcribe) it with symbols out of {encoder_alphabet_name}(alphabet).".format(
            message_category=self.message_category,
            encoder_alphabet_name=self.encoder_alphabet_name,
        ) #TODO: just cause seperator and ai doent like ach othwer

        """self.decode_description = "\nGiven a cipher as query decode (not necessarily transcribe) it back to a {message_category} with {decoder_alphabet_name}(alphabet). Divide symbols by: '{separator}".format(
            message_category=self.message_category,
            decoder_alphabet_name=self.decoder_alphabet_name,
            separator=self.separator,
        )"""

        self.decode_description = "\nGiven a cipher as query decode (not necessarily transcribe) it back to a {message_category} with {decoder_alphabet_name}(alphabet).".format(
            message_category=self.message_category,
            decoder_alphabet_name=self.decoder_alphabet_name,
        )

        self.validate_description = "TODO"
        # TODO. ask niklas if sepreator is alwys betwenn chars cause if "" is sepratot emojis and chars are complicated    promlem(if no seprator but alphabeat contain heart and bb)


@dataclass()
class RoundState:
    """
    Stores the intermediate and final results of a single example:
    the cipher and the decoded message(message prime).
    Class used for logging
    """

    def __init__(
        self, example: Example
    ):  # TODO: use, should prevent  seperate alphabet description querey trnsport
        self.example = example
        self.cipher = None
        self.ciphers = []
        self.message_prime = None
        self.message_primes = []


# ───────────────────────────────────
# Signature
# ───────────────────────────────────


class Signature(ABC):
    """Abstract base for all signatures."""

    @abstractmethod
    def check(
        self,
        query: QueryType,
        alphabet: AlphabetType,
    ) -> bool:
        """Check whether the query is valid given the alphabet."""
        pass


class WordsSignature(Signature, ABC):
    """Signature allowing arbitrary combinations of symbols."""

    NAME = "Valid is any word over the alphabet"
    pass


class NWordsSignature(Signature, ABC):
    """Signature restricting words to maximum length n."""

    NAME = "Valid is any word over the alphabet of length n"
    pass


class SingleChoiceSignature(Signature, ABC):
    """Signature restricted to a single symbol."""

    NAME = "Valid is any single choice of the alphabet"
    pass


class MultipleChoiceSignature(Signature, ABC):
    """Signature restricted to a fixed number of symbols."""

    NAME = "Valid are choices of the alphabet"
    pass


class PermutationsSignature(Signature, ABC):
    """Signature restricted to permutation of the entire alphabet."""

    NAME = "Valid is a permutation of the alphabet"
    pass


class VariationSignature(Signature, ABC):
    """Signature allowing variation of the alphabet."""

    NAME = "Valid is a variation of the alphabet"
    pass


# ───────────────────────────────────
# Adapter
# ───────────────────────────────────


class Adapter(ABC):
    """Interface for human or AI adapter interacting with the game."""

    @abstractmethod
    def present_data(
        self,
        query: QueryType,
        alphabet: AlphabetType,
        name: str,
    ) -> None:
        """Display the query and alphabet to the adapter."""
        pass

    @abstractmethod
    def get_data(self) -> QueryType:
        """Return the adapter's response (cipher or decoded message)."""
        pass


class HumanAdapter(Adapter):
    """adapter interacting with a human user."""

    pass


class AIAdapter(Adapter):
    """adapter interacting with an AI model."""

    pass


# ───────────────────────────────────
# Agent
# ───────────────────────────────────


class Agent(ABC):
    """Handles encoding and communication with an adapter."""

    def __init__(self, signature_cls: Type[Signature], adapter_cls: Type[Adapter]):
        self.signature = signature_cls()
        self.adapter = adapter_cls()

    @abstractmethod
    def ask_adapter_for_query(
        self, query: QueryType, alphabet: AlphabetType
    ) -> QueryType:
        """Run the adapter to produce a query."""
        pass

    @abstractmethod
    def verify_query_with_signature(
        self, query: QueryType, alphabet: AlphabetType
    ) -> bool:
        """Check if the produced query is valid."""
        pass


# ───────────────────────────────────
# GAME
# ───────────────────────────────────


class Game(ABC):
    """Game logic handling transmission, validation, and communication."""

    def __init__(
        self,
        task: Task,
        signature_encoder_cls: tuple[Type[Signature], Type[Adapter]],
        signature_decoder_cls: list[tuple[Type[Signature], Type[Adapter]]],
        signature_validator_cls: tuple[Type[Signature], Type[Adapter]],
        agent_cls: Type[Agent],
    ):
        self.task = task

        self.agent_sender = agent_cls(*signature_encoder_cls)

        self.agent_receivers = [agent_cls(*t) for t in signature_decoder_cls]

        self.agent_validator = agent_cls(*signature_validator_cls)

        self.points = {r: 0 for r in self.agent_receivers}

    @abstractmethod
    def call_agent(
        self, agent: Agent, query: QueryType, alphabet: AlphabetType
    ) -> tuple[bool, QueryType]:
        """Calls agent, returns a  Tuple of is_valid and query"""
        pass

    @abstractmethod
    def play_example(self, example: Example):
        pass

    @abstractmethod
    def play_task(self):
        """Play all examples in the task."""
        pass

In [25]:
# -----------------------------
# Signature
# -----------------------------


class MyWordsSignature(WordsSignature):
    """def check(self, query, alphabet) -> bool:
        query = (
            query.strip().replace("\n", "").replace("\r", "").split("-")
        )  # TODO - seperator
        return all(c in alphabet for c in query)"""
    
    def check(self, query, alphabet) -> bool:
        query = (
            query.strip().replace("\n", "").replace("\r", "")
        )
        return all(c in alphabet for c in regex.findall(r"\X", query))


# -----------------------------
# Adapter
# -----------------------------


class MyHumanAdapter(HumanAdapter):
    def present_data(self, query, alphabet, name, description):  # TODO signature
        # clear_output()
        print(description)
        print(f"\n{name}\nQuery: {query}\nAlphabet: {alphabet}")

    def get_data(self):
        query = input("\nEnter: ")
        print(f"\nYou typed: {query}")
        # time.sleep(3)
        return query  # todo: index als liste aus 3 symbolen


class MyAIAdapter(AIAdapter):

    def present_data(self, query, alphabet, name, description):

        query = "\n".join([description, f"\n{name}\nQuery: {query}\nAlphabet: {alphabet}", "Just return the answer, nothing else"])
        print(query)
        payload = build_multi_modal_payload(final_text=query)
        response = send_request(payload)

        global global_buffer
        global_buffer = response

    def get_data(self):

        query = get_message_text(global_buffer)
        print(f"\nYou typed: {query}")
        return query


# -----------------------------
# Agent
# -----------------------------


class MyAgent(Agent):

    def __init__(self, signature_cls, adapter_cls):
        super().__init__(signature_cls, adapter_cls)

    def ask_adapter_for_query(self, query, alphabet, description):
        self.adapter.present_data(
            query, alphabet, type(self.signature).NAME, description
        )
        return self.adapter.get_data()

    def verify_query_with_signature(self, query, alphabet):
        return self.signature.check(query, alphabet)


# -----------------------------
# Game
# -----------------------------


class MyGame(Game):  # TODO: ask Niklas, logic thats not game dependent to abstract

    def __init__(
        self,
        task,
        signature_encoder_cls,
        signature_decoder_cls,
        signature_validator_cls,
        agent_cls,
    ):
        super().__init__(
            task,
            signature_encoder_cls,
            signature_decoder_cls,
            signature_validator_cls,
            agent_cls,
        )

        # For simplicity, assume only one receiver # TODO: remove
        self.agent_receiver = self.agent_receivers[0]

    def call_agent(self, agent, query, alphabet, description):
        query = agent.ask_adapter_for_query(query, alphabet, description)
        success = agent.verify_query_with_signature(query, alphabet)

        return success, query

    def play_example(self, example):

        success, cipher = self.call_agent(
            agent=self.agent_sender,
            query=example.message,
            alphabet=example.encoder_alphabet,
            description=self.task.encode_description,
        )

        while not success:
            print("\nRetry")
            success, cipher = self.call_agent(
                self.agent_sender,
                example.message,
                example.encoder_alphabet,
                description=self.task.encode_description,
            )

        print("\nNext Phase")

        success, message_prime = self.call_agent(
            self.agent_receiver,
            cipher,
            example.decoder_alphabet,
            description=self.task.decode_description,
        )
        while not success:
            print("\nRetry")
            success, message_prime = self.call_agent(
                self.agent_receiver,
                cipher,
                example.decoder_alphabet,
                description=self.task.decode_description,
            )

        print("\nValidator Phase")

        success, points = self.call_agent(
            self.agent_validator,
            message_prime,
            example.validator_alphabet,
            description=self.task.validate_description,
        )

        while not success:
            print("\nRetry")
            success, points = self.call_agent(
                self.agent_validator,
                message_prime,
                example.validator_alphabet,
                description=self.task.validate_description,
            )

        # points = int(example.message.lower() == message_prime.lower())
        self.points[self.agent_receiver] += int(points)

        print(f"\nreceived points: {points}")

        # TODO: sleep

        print("\nNext Round")

    def play_task(self):
        for example in self.task.examples:
            self.play_example(example)

        print(f"\ntotal points: {self.points[self.agent_receiver]}")

In [26]:
if __name__ == "__main__":
    emoji_alphabet = [
        "❄️",
        "🛳️",
        "❤️",
        "💥",
        "🌊",
        "🧊",
        "💀",
        "🦈",
        "🚢",
        "🌹",
    ]  # a debugging symbol
    letter_alphabet = list("abcdefghijklmnopqrstuvwxyz")
    validator_alphabet = ["0", "1"]
    separator = ""
    description_format = ("movie title", "emoji list", "lower case letters", "Points")

    examples = [
        Example("Titanic", emoji_alphabet, letter_alphabet, validator_alphabet),
        # Example("Inception", emoji_alphabet, letter_alphabet, validator_alphabet),
        # Example("Avatar", emoji_alphabet, letter_alphabet, validator_alphabet),
    ]
    task = Task(examples, separator, description_format)

    game = MyGame(
        task=task,
        signature_encoder_cls=(MyWordsSignature, MyAIAdapter),
        signature_decoder_cls=[(MyWordsSignature, MyAIAdapter)],
        signature_validator_cls=(MyWordsSignature, MyHumanAdapter),
        agent_cls=MyAgent,
    )

    game.play_task()


Given a movie title as query, encode (not necessarily transcribe) it with symbols out of emoji list(alphabet).

Valid is any word over the alphabet
Query: Titanic
Alphabet: ['❄️', '🛳️', '❤️', '💥', '🌊', '🧊', '💀', '🦈', '🚢', '🌹']
Just return the answer, nothing else

You typed: 🚢🧊❤️💥


Next Phase

Given a cipher as query decode (not necessarily transcribe) it back to a movie title with lower case letters(alphabet).

Valid is any word over the alphabet
Query: 🚢🧊❤️💥

Alphabet: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Just return the answer, nothing else

You typed: star wars

Retry

Given a cipher as query decode (not necessarily transcribe) it back to a movie title with lower case letters(alphabet).

Valid is any word over the alphabet
Query: 🚢🧊❤️💥

Alphabet: ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']
Just return 

In [26]:

mws = MyWordsSignature()
mws.check(query=get_message_text(global_buffer), alphabet=emoji_alphabet)

True

In [20]:
# TODO:
# second game (4 Bilder, 1 Wort)
# human verifier, ask niklas
# code to .py, except main
# fix different signatures
# Seperator "" at unary letters
# Ranking given as list

In [21]:
# ──────────────────────────────
# N Words


class MyWordsEncoder(WordsEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols)


# ──────────────────────────────
# N Words
# ──────────────────────────────


class MyNWordsEncoder(NWordsEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols) and len(symbols) <= n


# -----------------------------
# SingleChoice
# -----------------------------
class MySingleChoiceEncoder(SingleChoiceEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols) and len(symbols) == 1


# -----------------------------
# MultipleChoice
# -----------------------------
class MyMultipleChoiceEncoder(MultipleChoiceEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return all(c in encoder_alphabet for c in symbols)


# -----------------------------
# Permutation
# -----------------------------
class MyPermutationsAllEncoder(PermutationsAllEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        return sorted(symbols) == sorted(encoder_alphabet)


# -----------------------------
# Variation
# -----------------------------
class MyPermutationsSubEncoder(PermutationsSubEncoder):
    def encode(self, cipher, encoder_alphabet) -> bool:
        symbols = regex.findall(r"\X", cipher)
        c1 = Counter(cipher)
        c2 = Counter(encoder_alphabet)
        return all(c1[x] <= c2[x] for x in c1) and len(c1) == len(encoder_alphabet)

NameError: name 'WordsEncoder' is not defined

In [ ]:
### chatbot methods


def load_local_image(path: Path) -> PImage.Image:
    """
    Load an image from disk as a PIL Image.
    """
    if not path.exists():
        raise FileNotFoundError(path)
    return PImage.open(path)


def encode_image_base64(image: PImage.Image) -> str:
    """
    Encode a PIL Image as base64 JPEG.
    """
    buffer = BytesIO()
    image.convert("RGB").save(buffer, format="JPEG")
    return base64.b64encode(buffer.getvalue()).decode("utf-8")


def get_message_text(response) -> str:
    return response.choices[0].message.content or ""


def send_request(payload: list[dict[str, any]]) -> str:  # TODO typehints
    """
    Send a single multimodal payload to the OpenAI-compatible chat API
    and return only the assistant's textual response.
    """

    client = OpenAI(
        base_url="https://chat.web.webis.de/openai/",  # "https://chat.web.webis.de/openai/" - with key or "https://llm.srv.webis.de/openai/v1/" in VPN
        api_key="sk-",
    )

    response = client.chat.completions.create(
        model="gemma3-4b",
        messages=[
            {
                "role": "user",
                "content": payload,
            }
        ],
        max_tokens=300,
    )

    return response


def build_multi_modal_payload(
    final_text: str,
    image_text_pairs: list[tuple[PImage.Image, str]] = [],
) -> list[dict[str, any]]:
    """
    Build a multimodal payload from optional image-text pairs and a final instruction.
    """

    payload  = []

    # Add image-text pairs if any
    for image, text in image_text_pairs:
        payload.append({"type": "text", "text": text})
        payload.append(
            {
                "type": "image_url",
                "image_url": {
                    "url": f"data:image/jpeg;base64,{encode_image_base64(image)}"
                },
            }
        )

    # Always add the final text
    payload.append({"type": "text", "text": final_text})

    return payload

In [ ]:
query = "Given a movie title as query, encode (not necessarily transcribe) it with symbols out of emoji list(alphabet). Divide symbols by: '-'\n\nValid is any word over the alphabet\nQuery: Titanic\nAlphabet: ['❄️', '🛳️', '❤️', '💥', '🌊', '🧊', '💀', '🦈', '🚢', '🌹']"

img_a = PImage.open("images/logoA.jpg")
img_b = PImage.open("images/logoB.jpg")

payload = build_multi_modal_payload(
    #image_text_pairs=[
    #    (img_a, "Logo A:"),
    #    (img_b, "Logo B:"),
    #],
    #final_text="Which logo is better and why?",
    final_text = query,
)

response = send_request(payload)
answer_text = get_message_text(response)
print(answer_text)


❄️-🛳️-❤️-💥-🌊-🧊-💀-🦈-🚢-🌹

